In [4]:
from IPython.display import HTML

In [5]:
#configure dask client 
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/home/suv/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://172.27.9.105/30365/1 Dashboard: http://172.27.9.105/30365/1:34303/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [6]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [7]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index

    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))

df.to_csv('data/*.csv', name_function=name);

In [8]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [9]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1029,Xavier,-0.7604221243407709,0.8401663916983746
2000-01-01 00:00:01,995,Hannah,0.7802390915276805,-0.23291123835451333
2000-01-01 00:00:02,1035,Patricia,-0.7904401259387535,0.026785590541341886
2000-01-01 00:00:03,1081,Jerry,0.8569381504157345,0.9154685235836701
2000-01-01 00:00:04,992,Dan,0.4411611109680569,-0.7662815137654369
2000-01-01 00:00:05,1012,Bob,-0.5639269181124376,-0.8873868384817809
2000-01-01 00:00:06,1001,Dan,-0.5251239963990777,-0.38631136515511355
2000-01-01 00:00:07,1020,Frank,0.18156726498069053,0.9408994670238471
2000-01-01 00:00:08,975,Oliver,-0.8038336514507463,-0.00835601986372203


####  PANDAS

In [10]:
%time
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1029,Xavier,-0.760422,0.840166
1,2000-01-01 00:00:01,995,Hannah,0.780239,-0.232911
2,2000-01-01 00:00:02,1035,Patricia,-0.790440,0.026786
3,2000-01-01 00:00:03,1081,Jerry,0.856938,0.915469
4,2000-01-01 00:00:04,992,Dan,0.441161,-0.766282


#### DASK

In [16]:
%time 
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [13]:
%time
df.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1029,Xavier,-0.760422,0.840166
1,2000-01-01 00:00:01,995,Hannah,0.780239,-0.232911
2,2000-01-01 00:00:02,1035,Patricia,-0.790440,0.026786
3,2000-01-01 00:00:03,1081,Jerry,0.856938,0.915469
4,2000-01-01 00:00:04,992,Dan,0.441161,-0.766282


In [22]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.69 s, sys: 358 ms, total: 4.05 s
Wall time: 2.73 s


name
Alice       0.000703
Bob         0.000516
Charlie     0.000888
Dan         0.000526
Edith      -0.000035
Frank      -0.000999
George      0.003643
Hannah      0.000680
Ingrid      0.002918
Jerry       0.001568
Kevin       0.001007
Laura       0.001736
Michael     0.001395
Norbert    -0.002855
Oliver      0.000934
Patricia   -0.001352
Quinn       0.000865
Ray        -0.000889
Sarah       0.002674
Tim         0.000928
Ursula      0.001028
Victor     -0.003110
Wendy      -0.000539
Xavier     -0.000948
Yvonne      0.002319
Zelda      -0.000293
Name: x, dtype: float64

In [21]:
import dask.dataframe as dd

In [26]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.68 s, sys: 373 ms, total: 4.05 s
Wall time: 2.7 s


name
Alice       0.000703
Bob         0.000516
Charlie     0.000888
Dan         0.000526
Edith      -0.000035
Frank      -0.000999
George      0.003643
Hannah      0.000680
Ingrid      0.002918
Jerry       0.001568
Kevin       0.001007
Laura       0.001736
Michael     0.001395
Norbert    -0.002855
Oliver      0.000934
Patricia   -0.001352
Quinn       0.000865
Ray        -0.000889
Sarah       0.002674
Tim         0.000928
Ursula      0.001028
Victor     -0.003110
Wendy      -0.000539
Xavier     -0.000948
Yvonne      0.002319
Zelda      -0.000293
Name: x, dtype: float64

####  GroupBy 

In [28]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [30]:
%time
df = df.persist()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [31]:
%time
df.groupby('name').x.mean().compute()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


name
Alice       0.000106
Bob        -0.000007
Charlie     0.000193
Dan         0.001348
Edith       0.001636
Frank       0.001063
George     -0.000276
Hannah      0.001875
Ingrid     -0.001356
Jerry       0.002244
Kevin       0.001058
Laura      -0.002503
Michael    -0.001484
Norbert    -0.000343
Oliver     -0.001190
Patricia    0.000072
Quinn       0.000594
Ray        -0.001443
Sarah      -0.001262
Tim        -0.001582
Ursula      0.000397
Victor      0.001039
Wendy      -0.000356
Xavier     -0.001460
Yvonne     -0.002091
Zelda       0.002794
Name: x, dtype: float64

In [33]:
df.describe

<bound method _Frame.describe of Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks>

In [35]:
%time 
df.count

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.39 µs


<bound method _Frame.count of Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks>

In [32]:
%time _ = df.groupby('id').x.mean().compute()

CPU times: user 276 ms, sys: 17.2 ms, total: 293 ms
Wall time: 301 ms


In [36]:
%time _ = df.groupby('name').x.mean().compute()

CPU times: user 546 ms, sys: 21.6 ms, total: 567 ms
Wall time: 410 ms


In [37]:
%time 
df.groupby('name').agg({'x': ['mean', 'std'], 'y': ['mean', 'count']}).compute().head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


x                   y       
             mean       std      mean  count
name                                        
Alice    0.000106  0.577363  0.002861  99695
Bob     -0.000007  0.577857  0.000456  99902
Charlie  0.000193  0.577482  0.000351  99280
Dan      0.001348  0.578629 -0.000946  99298
Edith    0.001636  0.578053 -0.001377  99381

In [38]:
%%time 
df.groupby('name').x.mean().visualize(node_attr={'penwidth': '6'})

RuntimeError: Drawing dask graphs requires the `graphviz` python library and the `graphviz` system library to be installed.

In [39]:
from sklearn.linear_model import LinearRegression

def train(partition):
    est = LinearRegression()
    est.fit(partition[['x', 'id']].values, partition.y.values)
    return est

In [40]:
%time 
df.groupby('name').apply(train, meta=object).compute().sort_index()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


name
Alice       LinearRegression(copy_X=True, fit_intercept=Tr...
Bob         LinearRegression(copy_X=True, fit_intercept=Tr...
Charlie     LinearRegression(copy_X=True, fit_intercept=Tr...
Dan         LinearRegression(copy_X=True, fit_intercept=Tr...
Edith       LinearRegression(copy_X=True, fit_intercept=Tr...
Frank       LinearRegression(copy_X=True, fit_intercept=Tr...
George      LinearRegression(copy_X=True, fit_intercept=Tr...
Hannah      LinearRegression(copy_X=True, fit_intercept=Tr...
Ingrid      LinearRegression(copy_X=True, fit_intercept=Tr...
Jerry       LinearRegression(copy_X=True, fit_intercept=Tr...
Kevin       LinearRegression(copy_X=True, fit_intercept=Tr...
Laura       LinearRegression(copy_X=True, fit_intercept=Tr...
Michael     LinearRegression(copy_X=True, fit_intercept=Tr...
Norbert     LinearRegression(copy_X=True, fit_intercept=Tr...
Oliver      LinearRegression(copy_X=True, fit_intercept=Tr...
Patricia    LinearRegression(copy_X=True, fit_intercept=Tr...
Qui